In [ ]:
library(DESeq2)
library(DEGreport)
library(tidyverse)
library(patchwork)

In [ ]:
readcount <- read.table('./source_data/batch_correction/batch_cor_GeneCount_mfa.txt',header=T)
rownames(readcount) <- readcount$gene_chr
readcount <- readcount[,c(2:25)]

In [ ]:
expdesign <- read.table("./source_data/reference/mfa_samplelist.txt",row.names = 1)

expdesign <- expdesign[c(1,2,9,10,17,18,3,4,11,12,19,20,5,6,13,14,21,22,7,8,15,16,23,24),]
expdesign$condition = factor(c(rep('G1',6),rep('G2',6),rep('G3',6),rep('G4',6)))

readcount <- readcount[,rownames(expdesign)]

In [ ]:
##############
# LRT method #
##############

dds <- DESeqDataSetFromMatrix(countData = readcount,
                              colData = expdesign,
                              design = ~ condition)

dds_LRT <- DESeq(dds, test="LRT", reduced = ~1)
res_LRT <-results(dds_LRT)

count <- counts(dds_LRT, normalized = TRUE)

basemean <- assay(rlog(dds_LRT, blind=FALSE))

In [ ]:
write.table(count, file='./source_data/DEmRNA/batch_cor_Gene_LRT_normalized_count_TC_mfa.txt', sep='\t',col.names = T, row.names = T)

In [ ]:
####################################
# analyzing patterns of expression #
####################################

sig_res_LRT <- res_LRT %>%
  data.frame() %>%
  rownames_to_column(var="gene") %>% 
  as_tibble() %>% 
  filter(padj < 0.001)

sig_res_LRT <- sig_res_LRT %>%
  arrange(padj)

sigLRT_genes <- sig_res_LRT %>% 
  pull(gene)

cluster_rlog <- basemean[sigLRT_genes, ]

In [ ]:
################
# pattern plot #
################

clusters <- degPatterns(cluster_rlog, metadata = expdesign, time = "condition", minc =15 ,reduce=TRUE)
write.table(clusters$normalized, file='./source_data/DEmRNA/batch_cor_TC_pattern_normalized_value.txt', sep='\t',col.names = T, row.names = F)

## Fig. 3b

In [ ]:
################
# pattern plot #
################

clusters_patterns <- clusters
options(repr.plot.width = 40,repr.plot.height =10)

for(i in unique(clusters_patterns$normalized$cluster)){
    clusters_group <- clusters_patterns$normalized[clusters_patterns$normalized$cluster == i,]
    N <- nrow(clusters_group)/4
    Plt = ggplot(clusters_group, aes(x = condition, y = value)) + theme_bw()+ coord_fixed(ratio = 0.9) +
    theme(panel.grid.major = element_blank(),panel.grid.major.x = element_line(),panel.grid.major.y = element_line())+
    geom_boxplot(aes(color=condition),alpha = 0.5,width = 0.3, outlier.shape = NA, size = 1)+
    geom_point(aes(color=condition),shape = 20, size =5, alpha = 0.7 ,position = position_jitter(0.05))+
    geom_line(aes(group=genes,color=condition),size = 0.1, alpha = 0.1)+
    geom_text(x=1.2, y=2, label=paste0("n=",N),size=15)+
    theme(axis.text.y= element_text(lineheight = 1, size = 50),
          axis.text.x= element_text(size = 50),
          axis.line.y = element_line(size = 1),
          axis.line.x = element_line(size = 1),
          axis.title = element_blank(),
          panel.border = element_blank(),
          panel.grid.major = element_blank(),
          panel.grid.minor = element_blank(),
          axis.line = element_line(colour = "black"),
          plot.title = element_text(size = 50))+
    theme(legend.position="none") + ggtitle(paste0("Pattern",i)) + ggeasy::easy_center_title()+
    scale_y_continuous(breaks=c(-1.5,0,1.5),limits=c(-2, 2))

    #png(paste0("/home/chm/analyzing_file_byR/RNA/SQuIRE_count_ELTC/figures/BC_Gene",i,"_re_group.png"),width=600,height=600,res=600)
    #options(repr.plot.width = 40,repr.plot.height =40)
    print(Plt)
    #dev.off()
}


In [ ]:
################
# parsing data #
################
cluster_groups <- clusters$df

#data <- cbind(res_LRT,basemean)
#data <- as.data.frame(data)

data <- as.data.frame(res_LRT)

data1 <- merge(cluster_groups,data,by.x = 0, by.y = 0)
split_result = str_split(data1$genes,"_", 2)
data1$gene = lapply(X = split_result, FUN = "[", 1)
data1$gene <- as.character(data1$gene)

write.table(data1[data1$cluster=="1",][,c(5,9,10)],file='./source_data/DEmRNA/group1_padj_score.txt',sep='\t',col.names = T,row.names = F)
write.table(data1[data1$cluster=="2",][,c(5,9,10)],file='./source_data/DEmRNA/group2_padj_score.txt',sep='\t',col.names = T,row.names = F)
write.table(data1[data1$cluster=="3",][,c(5,9,10)],file='./source_data/DEmRNA/group3_padj_score.txt',sep='\t',col.names = T,row.names = F)
write.table(data1[data1$cluster=="4",][,c(5,9,10)],file='./source_data/DEmRNA/group4_padj_score.txt',sep='\t',col.names = T,row.names = F)
write.table(data1[data1$cluster=="5",][,c(5,9,10)],file='./source_data/DEmRNA/group5_padj_score.txt',sep='\t',col.names = T,row.names = F)
write.table(data1[data1$cluster=="6",][,c(5,9,10)],file='./source_data/DEmRNA/group6_padj_score.txt',sep='\t',col.names = T,row.names = F)
write.table(data1[data1$cluster=="7",][,c(5,9,10)],file='./source_data/DEmRNA/group7_padj_score.txt',sep='\t',col.names = T,row.names = F)

In [ ]:
#############################################
#WGCNA input                            #####
#############################################
readcount_WGCNA <- readcount %>% dplyr::filter(rowSums(.) >= 50)
print(dim(readcount_WGCNA))
dds <- DESeqDataSetFromMatrix(countData = readcount_WGCNA,
                              colData = expdesign,
                              design = ~ condition)

wgcna_input <- assay(vst(dds, blind=FALSE))
wgcna_input <- cbind(rownames(wgcna_input),wgcna_input)
colnames(wgcna_input)[1] <- 'ensembl_id'
write.table(wgcna_input, file='./source_data/WGCNA/batch_cor_wgcna_input.txt', sep='\t',col.names = T, row.names = F)

## ********************************************

## Run 4_R_genecount_WGCNA.ipynb

## ********************************************

## TC_WGCNA

In [ ]:
wgcna_id_greenyellow <- read.table("./source_data/WGCNA/DEmRNA_age_greenyellow_WGCNA.txt",sep="\t")
wgcna_id_turquoise <- read.table("./source_data/WGCNA/DEmRNA_age_turquoise_WGCNA.txt",sep="\t")
wgcna_id <- do.call("rbind",list(wgcna_id_greenyellow,wgcna_id_turquoise))

In [ ]:
################
# parsing data #
################

clusters_wgcna <- clusters$normalized
WGCNA <- subset(clusters_wgcna,clusters_wgcna$genes %in% wgcna_id$V1)

cluster_groups <- clusters$df
cluster_groups <- cluster_groups[unique(WGCNA$genes),]

data2 <- merge(cluster_groups,data,by.x = 0, by.y = 0)
split_result = str_split(data2$genes,"_", 2)
data2$gene = lapply(X = split_result, FUN = "[", 1)
data2$gene <- as.character(data2$gene)

write.table(data2[data2$cluster=="1",][,c(5,9,10)],file='./source_data/DEmRNA/WGCNA_group1_padj_score.txt',sep='\t',col.names = T,row.names = F)
write.table(data2[data2$cluster=="2",][,c(5,9,10)],file='./source_data/DEmRNA/WGCNA_group2_padj_score.txt',sep='\t',col.names = T,row.names = F)
write.table(data2[data2$cluster=="3",][,c(5,9,10)],file='./source_data/DEmRNA/WGCNA_group3_padj_score.txt',sep='\t',col.names = T,row.names = F)
write.table(data2[data2$cluster=="4",][,c(5,9,10)],file='./source_data/DEmRNA/WGCNA_group4_padj_score.txt',sep='\t',col.names = T,row.names = F)
write.table(data2[data2$cluster=="5",][,c(5,9,10)],file='./source_data/DEmRNA/WGCNA_group5_padj_score.txt',sep='\t',col.names = T,row.names = F)
write.table(data2[data2$cluster=="6",][,c(5,9,10)],file='./source_data/DEmRNA/WGCNA_group6_padj_score.txt',sep='\t',col.names = T,row.names = F)
write.table(data2[data2$cluster=="7",][,c(5,9,10)],file='./source_data/DEmRNA/WGCNA_group7_padj_score.txt',sep='\t',col.names = T,row.names = F)